In [17]:
!pip install cohere

Defaulting to user installation because normal site-packages is not writeable


In [18]:
import pandas as pd
import cohere
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import time
import os

In [ ]:
co = cohere.Client("YOUR_COHERE_API_KEY")  

In [ ]:
df = pd.read_csv(r"ATTACH_YOUR_DATASET_PATH")
df.head()

,Text,Label
0,David Blunkett in quotes\n \n David Blunkett -...,0
1,Benitez issues warning to Gerrard\n \n Liverpo...,1
2,Brookside creator's Channel 4 bid\n \n The cre...,3
3,Brown visits slum on Africa trip\n \n Chancell...,0
4,Gritty return for Prince of Persia\n \n Still ...,2


In [ ]:
def get_embedding(text):
    while True:
        try:
            response = co.embed(
                texts=[text],
                model="embed-english-v3.0",
                input_type="search_document",  
                truncate="END"
            )
            return response.embeddings[0]
        except Exception as e:
            print(f"❌ Error: {e}. Retrying in 2s...")
            time.sleep(2)

In [22]:
embedding_file = "cohere_embeddings.pkl"

if os.path.exists(embedding_file):
    print("✅ Loaded cached embeddings")
    df = pd.read_pickle(embedding_file)
else:
    print("🚀 Generating Cohere embeddings...")
    df["embedding"] = df["Text"].apply(get_embedding)
    df.to_pickle(embedding_file)
    print(f"💾 Saved to {embedding_file}")


🚀 Generating Cohere embeddings...
💾 Saved to cohere_embeddings.pkl


In [23]:
embedding_matrix = np.vstack(df["embedding"].values)

def find_similar_entries(query, top_n=5):
    query_embedding = get_embedding(query)
    query_embedding = np.array(query_embedding).reshape(1, -1)
    similarities = cosine_similarity(query_embedding, embedding_matrix)[0]
    top_indices = similarities.argsort()[::-1][:top_n]
    return df.iloc[top_indices][["Text", "Label"]].assign(Similarity=similarities[top_indices])


In [24]:
query = input("🔍 Enter your query: ")
results = find_similar_entries(query)
display(results)


,Text,Label,Similarity
13,Schools to take part in mock poll\n \n Record ...,0,0.579614
48,Student 'fee factor' played down\n \n A rise h...,0,0.256741
34,Labour's election love-in\n \n Peace and love ...,0,0.219050
22,Lib Dems unveil election slogan\n \n The Liber...,0,0.216986
14,"A November to remember\n \n Last Saturday, one...",1,0.209654
